In [7]:
import pandas as pd
from sodapy import Socrata
from bs4 import BeautifulSoup
import requests
import numpy as np



In [8]:
client = Socrata("data.seattle.gov", None)
results = client.get("6vkj-f5xf", limit = 100)
results_df = pd.DataFrame.from_records(results)

In [13]:
new = results_df['title'].str.split(" /",n=1,expand = True)
results_df["short title"]= new[0]
results_df.head()

,author,bibnum,floatingitem,isbn,itemcollection,itemcount,itemlocation,itemtype,publicationyear,publisher,reportdate,subjects,title,short title
0,"O'Ryan, Ellie",3011076,Floating,"1481425730, 1481425749, 9781481425735, 9781481...",ncrdr,1,qna,jcbk,2014.,"Simon Spotlight,",2017-09-01T00:00:00.000,"Musicians Fiction, Bullfighters Fiction, Best ...",A tale of two friends / adapted by Ellie O'Rya...,A tale of two friends
1,"Kishimoto, Masashi, 1974-",2248846,NA,1569319006,nycomic,1,lcy,acbk,"2003, c1999.","Viz,",2017-09-01T00:00:00.000,"Ninja Japan Comic books strips etc, Comic book...","Naruto. Vol. 1, Uzumaki Naruto / story and art...","Naruto. Vol. 1, Uzumaki Naruto"
2,"Scott, Jerry, 1955-",3209270,NA,"144945867X, 9781449458676",nycomic,1,bea,acbk,2014.,"Andrews McMeel Publishing,",2017-09-01T00:00:00.000,Duncan Jeremy Fictitious character Comic books...,"Peace, love & Wi-Fi : a ZITS treasury / by Jer...","Peace, love & Wi-Fi : a ZITS treasury"
3,"Carlile, Clancy, 1930-",1907265,NA,0786706155,cafic,1,cen,acbk,c1999.,"Carroll & Graf,",2017-09-01T00:00:00.000,"Hemingway Ernest 1899 1961 Fiction, Biographic...",The Paris pilgrims : a novel / Clancy Carlile.,The Paris pilgrims : a novel
4,NaN,1644616,NA,094020813X,canf,1,cen,acbk,"1991, c1988.","Red Alder Books/Down There Press,",2017-09-01T00:00:00.000,"Erotic literature American, American literatur...","Erotic by nature : a celebration of life, of l...","Erotic by nature : a celebration of life, of l..."


In [11]:
#You can chain startswith and endswith masks or use contains - ^ is for match start of string, .* is for any string and $ for end:

In [36]:
base_url = "https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_"

year_range = np.arange(1971, 2018, dtype=int)
#year_range = [2018]
print(year_range)

[1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984
 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015 2016 2017]


In [ ]:
# append to one data frame
year = base_url + str(1970)
tables = pd.read_html(year)
df_prime = tables[0]
new_header = df_prime.iloc[0] #grab the first row for the header
df_prime = df_prime[1:] #take the data less the header row
df_prime.columns = new_header #set the header row as the df header
df_prime = df_prime.dropna()
df_prime['Year'] = 1970

for y in year_range:

    year = base_url + str(y)
    tables = pd.read_html(year)
    df = tables[0]
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    df = df.dropna()
    df['Year'] = y
    df_prime.append(df)
        
       
    

In [38]:
df_prime.head(100)

,Date,Book,Author,Year
1,January 4,The Godfather,Mario Puzo,1970
6,February 8,The French Lieutenant's Woman,John Fowles,1970
7,February 15,The Godfather,Mario Puzo,1970
8,February 22,The French Lieutenant's Woman,John Fowles,1970
19,May 10,Love Story,Erich Segal,1970
